In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=133
)
 
print(f"Using backend {backend.name}")

Using backend ibm_torino


In [3]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci, ao2mo

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.problems import ElectronicStructureProblem

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator
from qiskit_ibm_runtime import EstimatorV2

# ----- Configuration -----
BASIS_SET = "sto-3g"

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of O2 using a manually constructed
    p-orbital active space (8e, 6o) for a 12-qubit simulation.
    """
    print(f"O₂ Ground State Calculation for Basis: {BASIS_SET}")
    print("Using a manually constructed (8, 6) p-orbital active space\n")

    atom_str = "O 0.0 0.0 0.0; O 0.0 0.0 1.21"
    O2_SPIN = 2
    results = {}

    # --- 1. Classical Calculation (Full System) ---
    print("--- Running Full System Classical Calculation ---")
    start_time = time.time()
    mol = gto.M(atom=atom_str, basis=BASIS_SET, spin=O2_SPIN, verbose=0)
    mf_uhf = scf.UHF(mol).run()
    results['FCI_Full'] = fci.FCI(mol, mf_uhf.mo_coeff).kernel()[0]
    print(f"Full System FCI (Exact) Energy: {results['FCI_Full']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Manual Problem Construction (Bypassing Transformers) ---
    print("--- Manually Constructing the Active Space Problem ---")
    
    n_orbitals = mf_uhf.mo_coeff[0].shape[1]

    # CHANGED: Define a smaller active space for a 12-qubit problem
    # We now have 4 inactive orbitals (2 core + 2 low-lying valence)
    n_inactive_orbitals = 4
    n_active_orbitals = 6  # This will result in 12 qubits
    active_orbital_indices = list(range(n_inactive_orbitals, n_inactive_orbitals + n_active_orbitals))
    
    # CHANGED: Manually define the number of active electrons for this space
    # This corresponds to the (5 alpha, 3 beta) occupation of the p-shell
    n_active_electrons_alpha = 5
    n_active_electrons_beta = 3
    
    # Get and transform electronic integrals from PySCF
    h1_ao = mf_uhf.get_hcore()
    h2_ao = ao2mo.restore(1, mf_uhf._eri, n_orbitals)
    mo_a, mo_b = mf_uhf.mo_coeff
    h1_mo_a = mo_a.T @ h1_ao @ mo_a
    h1_mo_b = mo_b.T @ h1_ao @ mo_b
    h2_mo_aa = ao2mo.kernel(h2_ao, mo_a)
    h2_mo_bb = ao2mo.kernel(h2_ao, mo_b)
    h2_mo_ab = ao2mo.kernel(h2_ao, (mo_a, mo_a, mo_b, mo_b))

    # Select only the integrals corresponding to our active orbitals
    idx = np.ix_(active_orbital_indices, active_orbital_indices)
    h1_active_a = h1_mo_a[idx]
    h1_active_b = h1_mo_b[idx]
    idx_4 = np.ix_(active_orbital_indices, active_orbital_indices, 
                   active_orbital_indices, active_orbital_indices)
    h2_active_aa = h2_mo_aa[idx_4]
    h2_active_bb = h2_mo_bb[idx_4]
    h2_active_ab = h2_mo_ab[idx_4]

    # Manually create the electronic energy Hamiltonian
    electronic_hamiltonian = ElectronicEnergy.from_raw_integrals(
        h1_active_a, h2_active_aa, h1_active_b, h2_active_bb, h2_active_ab
    )
    electronic_hamiltonian.nuclear_repulsion_energy = mol.energy_nuc()
    
    # Manually create the final problem instance
    problem_final = ElectronicStructureProblem(electronic_hamiltonian)
    problem_final.num_spatial_orbitals = n_active_orbitals
    problem_final.num_particles = (n_active_electrons_alpha, n_active_electrons_beta)
    
    print(f"Final problem (manual): {problem_final.num_particles} electrons in {problem_final.num_spatial_orbitals} orbitals.")
    print("-" * 50)
    
    # --- 3. Quantum Calculation (VQE) ---
    print("\n--- Running VQE on the 12-Qubit Problem ---")
    start_time = time.time()
    
    mapper = JordanWignerMapper()
    ansatz = UCCSD(
        problem_final.num_spatial_orbitals,
        problem_final.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem_final.num_spatial_orbitals,
            problem_final.num_particles,
            mapper,
        ),
    )
    
    estimator = EstimatorV2(backend=backend, service=service)
    vqe_instance = VQE(estimator, ansatz, SLSQP())
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem_final)
    results['VQE_Transformed'] = result_vqe.total_energies[0]
    
    hamiltonian_op = problem_final.second_q_ops()[0]
    qubit_op = mapper.map(hamiltonian_op)
    print(f"Number of qubits required: {qubit_op.num_qubits}")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("           Final Energy Summary for O₂")
    print("="*50)
    print(f"  VQE (12-Qubit) Energy: {results['VQE_Transformed']:.8f} Hartree")
    print(f"  FCI (Full System) Energy: {results['FCI_Full']:.8f} Hartree")
    print("-" * 50)
    error = abs(results['VQE_Transformed'] - results['FCI_Full'])
    print(f"  Error (VQE vs Full FCI):  {error:.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()

ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (/config/miniconda3/envs/vqe/lib/python3.11/site-packages/qiskit/primitives/__init__.py)

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from typing import Sequence

# Qiskit components
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives.base import BaseEstimatorV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Nature for molecular simulation
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

# Qiskit IBM Runtime for running on hardware
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator

# ====================================================================
# 1. DEFINE THE MOLECULAR PROBLEM
# ====================================================================
molecule_string = "H 0 0 0; H 0 0 0.735"
driver = PySCFDriver(atom=molecule_string, basis="sto3g")
problem = driver.run()

# Mapper
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.hamiltonian.second_q_op())
num_qubits = qubit_op.num_qubits

# Get exact FCI energy directly from PySCF
from pyscf import gto, scf, fci
mol = gto.Mole()
mol.build(atom=molecule_string, basis="sto3g")
mf = scf.RHF(mol).run()
cisolver = fci.FCI(mol, mf.mo_coeff)
fci_energy = cisolver.kernel()[0]
fci_result = fci_energy + mol.energy_nuc()
print(f"Classical FCI (exact) energy: {fci_result:.12f}")

# ====================================================================
# 2. SELECT BACKEND AND PREPARE CIRCUITS
# ====================================================================
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, min_num_qubits=num_qubits, simulator=False
)
print(f"Using backend: {backend.name}")
target = backend.target

# Ansatz
ansatz = UCCSD(
    num_spatial_orbitals=problem.num_spatial_orbitals,
    num_particles=problem.num_particles,
    mapper=mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals, problem.num_particles, mapper
    ),
)

# Transpile
print("Transpiling circuit...")
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
ansatz_ibm = pm.run(ansatz)

# Observable with layout
observable_ibm = qubit_op.apply_layout(ansatz_ibm.layout)
print(f"Ansatz has {ansatz_ibm.num_parameters} parameters.")

# ====================================================================
# 3. DEFINE COST AND CALLBACK FUNCTIONS
# ====================================================================
def cost_func(
    params: Sequence,
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
) -> float:
    """Ground state energy evaluation."""
    pub = (ansatz, hamiltonian, [params])
    result = estimator.run([pub]).result()[0]
    return result.data.evs.real.item()

def build_callback(
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
    callback_dict: dict,
):
    def callback(current_vector):
        callback_dict["iters"] += 1
        callback_dict["prev_vector"] = current_vector
        
        pub = (ansatz, hamiltonian, [current_vector])
        result = estimator.run([pub]).result()[0]
        current_cost = result.data.evs.real.item()
        
        callback_dict["cost_history"].append(current_cost)
        print(
            f"Iters: {callback_dict['iters']:3d}  |  Energy: {current_cost:.12f}",
            end="\r",
            flush=True,
        )
    return callback

# ====================================================================
# 4. RUN THE VQE OPTIMIZATION
# ====================================================================
num_params = ansatz_ibm.num_parameters
initial_params = 2 * np.pi * np.random.random(num_params)

callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

print("\nStarting VQE run on real hardware. This may take a while...")
with Session(backend=backend) as session:
    estimator = Estimator(session=session)
    estimator.options.default_shots = 10000 
    estimator.options.resilience_level = 1

    callback = build_callback(
        ansatz_ibm, observable_ibm, estimator, callback_dict
    )
    
    res = minimize(
        cost_func,
        x0=initial_params,
        args=(ansatz_ibm, observable_ibm, estimator),
        callback=callback,
        method="cobyla",
        options={"maxiter": 100},
    )

print("\nOptimization complete.")

# ====================================================================
# 5. VISUALIZE AND PRINT RESULTS
# ====================================================================
print(f"\nEstimated ground state energy: {res.fun:.12f}")
print(f"Classical FCI (exact) energy: {fci_result:.12f}")
error = abs(res.fun - fci_result)
print(f"Absolute error: {error:.6f} Hartrees")

plt.plot(callback_dict["cost_history"], lw=2)
plt.axhline(y=fci_result, color='r', linestyle='--', label='FCI Energy')
plt.xlabel("Iteration")
plt.ylabel("Energy (Hartrees)")
plt.title("VQE Convergence for H₂ Molecule")
plt.legend()
plt.grid(True)
plt.show()


converged SCF energy = -1.116998996754
Classical FCI (exact) energy: -0.417337041304
Using backend: ibm_torino


TypeError: UCCSD.__init__() got an unexpected keyword argument 'mapper'

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from typing import Sequence

# Qiskit components
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives.base import BaseEstimatorV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Nature for molecular simulation
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

# Qiskit IBM Runtime for running on hardware
# Note: You need to have your IBM Quantum account configured.
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator

# ====================================================================
# 1. DEFINE THE MOLECULAR PROBLEM
# ====================================================================
molecule_string = "H 0 0 0; H 0 0 0.735"
driver = PySCFDriver(atom=molecule_string, basis="sto3g")
problem = driver.run()

# Mapper
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.hamiltonian.second_q_op())
num_qubits = qubit_op.num_qubits

# Get exact FCI energy directly from PySCF
from pyscf import gto, scf, fci
mol = gto.Mole()
mol.build(atom=molecule_string, basis="sto3g")
mf = scf.RHF(mol).run()
cisolver = fci.FCI(mol, mf.mo_coeff)
fci_energy = cisolver.kernel()[0]
fci_result = fci_energy + mol.energy_nuc()
print(f"Classical FCI (exact) energy: {fci_result:.12f}")

# ====================================================================
# 2. SELECT BACKEND AND PREPARE CIRCUITS
# ====================================================================
service = QiskitRuntimeService() # Uncomment to run on real hardware
backend = service.least_busy(
    operational=True, min_num_qubits=num_qubits, simulator=False
)
print(f"Using backend: {backend.name}")
# --- Using a local simulator for demonstration as hardware access is not available ---
# from qiskit.primitives import Estimator as LocalEstimator
# from qiskit_aer.primitives import Estimator as AerEstimator
# from qiskit_aer import AerSimulator
# backend = AerSimulator()
# print("Using local AerSimulator instead of real hardware for this example.")
# ---------------------------------------------------------------------------------


# Ansatz
# FIX 1: The 'mapper' argument in HartreeFock is now 'qubit_mapper'.
# FIX 2: The 'mapper' argument has been removed from the UCCSD constructor.
initial_state = HartreeFock(
    problem.num_spatial_orbitals,
    problem.num_particles,
    qubit_mapper=mapper  # FIX 1: Changed 'mapper' to 'qubit_mapper'
)
ansatz = UCCSD(
    num_spatial_orbitals=problem.num_spatial_orbitals,
    num_particles=problem.num_particles,
    initial_state=initial_state,
    qubit_mapper=mapper
)

# Transpile
print("Transpiling circuit...")
# Note: Transpilation is more relevant for real hardware.
# For a simulator, it's not strictly necessary but good practice.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
ansatz_ibm = pm.run(ansatz)

# Observable with layout
# For simulators, layout is trivial, but this is crucial for hardware
if hasattr(ansatz_ibm, 'layout') and ansatz_ibm.layout is not None:
    observable_ibm = qubit_op.apply_layout(ansatz_ibm.layout)
else:
    observable_ibm = qubit_op # No layout to apply for simulator
    
print(f"Ansatz has {ansatz_ibm.num_parameters} parameters.")

# ====================================================================
# 3. DEFINE COST AND CALLBACK FUNCTIONS
# ====================================================================
def cost_func(
    params: Sequence,
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
) -> float:
    """Ground state energy evaluation."""
    pub = (ansatz, hamiltonian, [params])
    result = estimator.run([pub]).result()[0]
    return result.data.evs.real.item()

def build_callback(
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
    callback_dict: dict,
):
    def callback(current_vector):
        callback_dict["iters"] += 1
        callback_dict["prev_vector"] = current_vector
        
        pub = (ansatz, hamiltonian, [current_vector])
        result = estimator.run([pub]).result()[0]
        current_cost = result.data.evs.real.item()
        
        callback_dict["cost_history"].append(current_cost)
        print(
            f"Iters: {callback_dict['iters']:3d}  |  Energy: {current_cost:.12f}",
            end="\r",
            flush=True,
        )
    return callback

# ====================================================================
# 4. RUN THE VQE OPTIMIZATION
# ====================================================================
num_params = ansatz_ibm.num_parameters
initial_params = 2 * np.pi * np.random.random(num_params)

callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

print("\nStarting VQE run on real hardware. This may take a while...")

with Session(backend=backend) as session:
    estimator = Estimator(session=session)
    # Set shots and resilience level for error mitigation
    estimator.options.default_shots = 10000
    estimator.options.resilience_level = 1

    callback = build_callback(
        ansatz_ibm, observable_ibm, estimator, callback_dict
    )

    res = minimize(
        cost_func,
        x0=initial_params,
        args=(ansatz_ibm, observable_ibm, estimator),
        callback=callback,
        method="cobyla",
        options={"maxiter": 100},
    )

print("\nOptimization complete.")

# ====================================================================
# 5. VISUALIZE AND PRINT RESULTS
# ====================================================================
print(f"\nEstimated ground state energy: {res.fun:.12f}")
print(f"Classical FCI (exact) energy: {fci_result:.12f}")
error = abs(res.fun - fci_result)
print(f"Absolute error: {error:.6f} Hartrees")

plt.figure(figsize=(10, 6))
plt.plot(callback_dict["cost_history"], lw=2, label="VQE Energy")
plt.axhline(y=fci_result, color='r', linestyle='--', label='FCI Exact Energy')
plt.xlabel("Iteration")
plt.ylabel("Energy (Hartrees)")
plt.title(f"VQE Convergence for H₂ Molecule on {backend.name}")
plt.legend()
plt.grid(True)
plt.show()



converged SCF energy = -1.116998996754
Classical FCI (exact) energy: -0.417337041304
Using backend: ibm_torino
Transpiling circuit...
Ansatz has 3 parameters.

Starting VQE run on real hardware. This may take a while...


RequestsApiError: '400 Client Error: Bad Request for url: https://quantum.cloud.ibm.com/api/v1/sessions. {"errors":[{"code":1352,"message":"You are not authorized to run a session when using the open plan.","solution":"Create an instance of a different plan type or use a different [execution mode](https://quantum.cloud.ibm.com/docs/guides/execution-modes).","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"770dcff6-381f-4c35-90d7-d5f9eb04b0cf"}\n'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from typing import Sequence

# Qiskit components
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives.base import BaseEstimatorV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Nature for molecular simulation
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

# Qiskit IBM Runtime for running on hardware
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

# ====================================================================
# 1. DEFINE THE MOLECULAR PROBLEM
# ====================================================================
molecule_string = "H 0 0 0; H 0 0 0.735"
driver = PySCFDriver(atom=molecule_string, basis="sto3g")
problem = driver.run()

# Mapper
mapper = JordanWignerMapper()
qubit_op = mapper.map(problem.hamiltonian.second_q_op())
num_qubits = qubit_op.num_qubits

# Get exact FCI energy directly from PySCF
from pyscf import gto, scf, fci
mol = gto.Mole()
mol.build(atom=molecule_string, basis="sto3g")
mf = scf.RHF(mol).run()
cisolver = fci.FCI(mol, mf.mo_coeff)
fci_energy = cisolver.kernel()[0]
fci_result = fci_energy + mol.energy_nuc()
print(f"Classical FCI (exact) energy: {fci_result:.12f}")

# ====================================================================
# 2. SELECT BACKEND AND PREPARE CIRCUITS
# ====================================================================
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, min_num_qubits=num_qubits, simulator=False
)
print(f"Using backend: {backend.name}")

# Ansatz
initial_state = HartreeFock(
    problem.num_spatial_orbitals,
    problem.num_particles,
    qubit_mapper=mapper
)
ansatz = UCCSD(
    num_spatial_orbitals=problem.num_spatial_orbitals,
    num_particles=problem.num_particles,
    initial_state=initial_state,
    qubit_mapper=mapper
)

# Transpile
print("Transpiling circuit...")
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
ansatz_ibm = pm.run(ansatz)

# Observable with layout
observable_ibm = qubit_op.apply_layout(ansatz_ibm.layout)
print(f"Ansatz has {ansatz_ibm.num_parameters} parameters.")

# ====================================================================
# 3. DEFINE COST AND CALLBACK FUNCTIONS
# ====================================================================
def cost_func(
    params: Sequence,
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
) -> float:
    """Ground state energy evaluation."""
    pub = (ansatz, hamiltonian, [params])
    result = estimator.run([pub]).result()[0]
    return result.data.evs.real.item()

def build_callback(
    ansatz: QuantumCircuit,
    hamiltonian: SparsePauliOp,
    estimator: BaseEstimatorV2,
    callback_dict: dict,
):
    def callback(current_vector):
        callback_dict["iters"] += 1
        callback_dict["prev_vector"] = current_vector

        pub = (ansatz, hamiltonian, [current_vector])
        result = estimator.run([pub]).result()[0]
        current_cost = result.data.evs.real.item()

        callback_dict["cost_history"].append(current_cost)
        print(
            f"Iters: {callback_dict['iters']:3d}  |  Energy: {current_cost:.12f}",
            end="\r",
            flush=True,
        )
    return callback

# ====================================================================
# 4. RUN THE VQE OPTIMIZATION
# ====================================================================
num_params = ansatz_ibm.num_parameters
initial_params = 2 * np.pi * np.random.random(num_params)

callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

print("\nStarting VQE run on real hardware. This may take a while...")

# FIX: Set shots using the 'default_shots' top-level property.
# The 'execution' dictionary is no longer used for this.
options = {
    "resilience_level": 1,
    "default_shots": 10000
}
estimator = Estimator(mode=backend, options=options)


callback = build_callback(
    ansatz_ibm, observable_ibm, estimator, callback_dict
)

res = minimize(
    cost_func,
    x0=initial_params,
    args=(ansatz_ibm, observable_ibm, estimator),
    callback=callback,
    method="cobyla",
    options={"maxiter": 100},
)

print("\nOptimization complete.")

# ====================================================================
# 5. VISUALIZE AND PRINT RESULTS
# ====================================================================
print(f"\nEstimated ground state energy: {res.fun:.12f}")
print(f"Classical FCI (exact) energy: {fci_result:.12f}")
error = abs(res.fun - fci_result)
print(f"Absolute error: {error:.6f} Hartrees")

plt.figure(figsize=(10, 6))
plt.plot(callback_dict["cost_history"], lw=2, label="VQE Energy")
plt.axhline(y=fci_result, color='r', linestyle='--', label='FCI Exact Energy')
plt.xlabel("Iteration")
plt.ylabel("Energy (Hartrees)")
plt.title(f"VQE Convergence for H₂ Molecule on {backend.name}")
plt.legend()
plt.grid(True)
plt.show()


converged SCF energy = -1.116998996754
Classical FCI (exact) energy: -0.417337041304
Using backend: ibm_torino
Transpiling circuit...
Ansatz has 3 parameters.

Starting VQE run on real hardware. This may take a while...


KeyboardInterrupt: 

In [ ]:
!pip list|grep qiskit

In [8]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry ---
from pyscf import gto, scf, fci

# --- Qiskit Nature ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

# --- Qiskit ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

# --- Config ---
BASIS_SET = "3-21g"
BOND_LENGTH = 0.8561

def main():
    print(f"H₃⁺ Ground State Calculation using IBM Quantum Runtime")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # === Molecule geometry ===
    r = BOND_LENGTH / np.sqrt(3)
    atom_str = (
        f"H {r:.6f} 0.0 0.0; "
        f"H {-r/2:.6f} {r * np.sqrt(3)/2:.6f} 0.0; "
        f"H {-r/2:.6f} {-r * np.sqrt(3)/2:.6f} 0.0"
    )

    results = {}

    # === Classical calculations ===
    print("--- Running Classical Calculations ---")
    start_time = time.time()

    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=1, spin=0, verbose=0)
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']} Hartree")

    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # === Qiskit Nature setup ===
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=1, spin=0)
    problem = driver.run()

    mapper = JordanWignerMapper()
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())

    print("Number of qubits:", qubit_op.num_qubits)

    hf_state = HartreeFock(
        num_spatial_orbitals=problem.num_spatial_orbitals,
        num_particles=problem.num_particles,
        qubit_mapper=mapper,
    )

    ansatz = UCCSD(
        num_spatial_orbitals=problem.num_spatial_orbitals,
        num_particles=problem.num_particles,
        qubit_mapper=mapper,
        initial_state=hf_state,
    )

    # === IBM Quantum Runtime connection ===
    service = QiskitRuntimeService()

    backend = service.least_busy(
        operational=True, simulator=False, min_num_qubits=qubit_op.num_qubits
    )
    print(f"Using backend: {backend.name}")

    options = Options()
    options.execution.shots = 8192

    optimizer = SLSQP(maxiter=200)

    with Session(service=service, backend=backend) as session:
        estimator = Estimator(session=session, options=options)

        vqe_instance = VQE(estimator=estimator, ansatz=ansatz, optimizer=optimizer)
        if hasattr(ansatz, "num_parameters"):
            vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

        print("--- Running VQE ---")
        start_time = time.time()
        result = vqe_instance.compute_minimum_eigenvalue(operator=qubit_op)
        results['VQE'] = result.eigenvalue.real
        print(f"VQE Energy: {results['VQE']} Hartree")
        print(f"VQE runtime: {time.time() - start_time:.2f}s\n")

    # === Summary ===
    print("\n" + "="*50)
    print("Final Energy Summary")
    print("="*50)
    print(f"  UHF Energy:           {results['UHF']} Hartree")
    print(f"  VQE Energy:           {results['VQE']} Hartree")
    print(f"  FCI (Exact) Energy:  {results['FCI']} Hartree")
    print("-" * 50)
    print(f"  Error (VQE vs FCI):  {abs(results['VQE'] - results['FCI'])} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()


ImportError: cannot import name 'Estimator' from 'qiskit.primitives' (/config/miniconda3/envs/end/lib/python3.13/site-packages/qiskit/primitives/__init__.py)